In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import base64
#!pip install spotipy
import requests
from bs4 import BeautifulSoup
import time

In [3]:
charts_df = pd.read_csv('../data/charts.csv')
lyrics_df = pd.read_csv('../data/billboard_24years_lyrics_spotify.csv')

In [4]:
charts_df.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


In [5]:
lyrics_df.head()

,ranking,song,band_singer,songurl,titletext,url,year,lyrics,uri,danceability,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,1,Breathe,Faith Hill,/wiki/Breathe_(Faith_Hill_song),Breathe,/wiki/Faith_Hill,2000,I can feel the magic floating in the air\nBein...,spotify:track:3y4LxiYMgDl4RethdzpmNe,0.529,...,0.000000,0.2510,0.278,136.859,audio_features,3y4LxiYMgDl4RethdzpmNe,https://api.spotify.com/v1/tracks/3y4LxiYMgDl4...,https://api.spotify.com/v1/audio-analysis/3y4L...,250547.0,4.0
1,2,Smooth,Santana,/wiki/Smooth_(Santana_song),Smooth,/wiki/Santana_(band),2000,"Man, it's a hot one\nLike seven inches from th...",spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,...,0.000005,0.2950,0.961,115.996,audio_features,0n2SEXB2qoRQg171q7XqeW,https://api.spotify.com/v1/tracks/0n2SEXB2qoRQ...,https://api.spotify.com/v1/audio-analysis/0n2S...,294987.0,4.0
2,2,Smooth,Rob Thomas,/wiki/Smooth_(Santana_song),Smooth,/wiki/Rob_Thomas_(musician),2000,"Man, it's a hot one\nLike seven inches from th...",spotify:track:5IALWUYK0zDSEmZgb4ICvc,0.590,...,0.807000,0.2990,0.724,115.983,audio_features,5IALWUYK0zDSEmZgb4ICvc,https://api.spotify.com/v1/tracks/5IALWUYK0zDS...,https://api.spotify.com/v1/audio-analysis/5IAL...,244924.0,4.0
3,3,Maria Maria,Santana,/wiki/Maria_Maria,Maria Maria,/wiki/Santana_(band),2000,"Ladies and gents, turn up your sound systems\n...",spotify:track:3XKIUb7HzIF1Vu9usunMzc,0.777,...,0.002010,0.0348,0.680,97.911,audio_features,3XKIUb7HzIF1Vu9usunMzc,https://api.spotify.com/v1/tracks/3XKIUb7HzIF1...,https://api.spotify.com/v1/audio-analysis/3XKI...,261973.0,4.0
4,3,Maria Maria,The Product G&B,/wiki/Maria_Maria,Maria Maria,/wiki/The_Product_G%26B,2000,Turn up this sound system\nTo the sound of Car...,spotify:track:3XKIUb7HzIF1Vu9usunMzc,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


client_id = 'client_id'
client_secret = 'client_secret'

# Encode credentials
auth_str = f"{client_id}:{client_secret}"
b64_auth_str = base64.b64encode(auth_str.encode()).decode()

# Request access token
auth_response = requests.post(
    'https://accounts.spotify.com/api/token',
    data={'grant_type': 'client_credentials'},
    headers={'Authorization': f'Basic {b64_auth_str}'}
)

# Extract token
access_token = auth_response.json()['access_token']
headers = {'Authorization': f'Bearer {access_token}'}

In [12]:
charts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date            330087 non-null  object 
 1   rank            330087 non-null  int64  
 2   song            330087 non-null  object 
 3   artist          330087 non-null  object 
 4   last-week       297775 non-null  float64
 5   peak-rank       330087 non-null  int64  
 6   weeks-on-board  330087 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 17.6+ MB


In [14]:
charts_df['artist'].value_counts()

artist
Taylor Swift                                      1023
Elton John                                         889
Madonna                                            857
Drake                                              787
Kenny Chesney                                      769
                                                  ... 
RiceGum Featuring Alissa Violet                      1
XXXTENTACION Featuring PnB Rock & Trippie Redd       1
Justin Bieber Featuring Burna Boy                    1
Justin Bieber Featuring BEAM                         1
Frankie Vaughan                                      1
Name: count, Length: 10205, dtype: int64

In [21]:
lyrics_df = lyrics_df.rename(columns={'ranking': 'rank'})
lyrics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3397 entries, 0 to 3396
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rank              3397 non-null   int64  
 1   song              3397 non-null   object 
 2   band_singer       3397 non-null   object 
 3   songurl           3397 non-null   object 
 4   titletext         3397 non-null   object 
 5   url               3397 non-null   object 
 6   year              3397 non-null   int64  
 7   lyrics            3397 non-null   object 
 8   uri               3397 non-null   object 
 9   danceability      486 non-null    float64
 10  energy            486 non-null    float64
 11  key               486 non-null    float64
 12  loudness          486 non-null    float64
 13  mode              486 non-null    float64
 14  speechiness       486 non-null    float64
 15  acousticness      486 non-null    float64
 16  instrumentalness  486 non-null    float64


In [43]:
charts_df['datetime'] = pd.to_datetime(charts_df['date'])
charts_df['year'] = charts_df['datetime'].dt.year

In [44]:
charts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            330087 non-null  object        
 1   rank            330087 non-null  int64         
 2   song            330087 non-null  object        
 3   artist          330087 non-null  object        
 4   last-week       297775 non-null  float64       
 5   peak-rank       330087 non-null  int64         
 6   weeks-on-board  330087 non-null  int64         
 7   datetime        330087 non-null  datetime64[ns]
 8   year            330087 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3), object(3)
memory usage: 21.4+ MB


In [51]:
charts_and_lyrics_df = pd.merge(charts_df, lyrics_df, on=['year', 'song'], how='inner')
pd.set_option('display.max_columns', None)
charts_and_lyrics_df

,date,rank_x,song,artist,last-week,peak-rank,weeks-on-board,datetime,year,rank_y,band_singer,songurl,titletext,url,lyrics,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,2021-11-06,2021,12,The Kid Laroi,/wiki/Stay_(The_Kid_Laroi_and_Justin_Bieber_song),Stay,/wiki/The_Kid_Laroi,I do the same thing I told you that I never wo...,spotify:track:567e29TDzLwZwfDuEpGTwo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,2021-11-06,2021,12,Justin Bieber,/wiki/Stay_(The_Kid_Laroi_and_Justin_Bieber_song),Stay,/wiki/Justin_Bieber,I do the same thing I told you that I never wo...,spotify:track:5HCyWlXZPP0y6Gqq8TgA20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,2021-11-06,2021,24,Lil Nas X,/wiki/Industry_Baby,Industry Baby,/wiki/Lil_Nas_X,"(D-D-Daytrip took it to ten, hey)\nBaby back, ...",spotify:track:27NovPIUIRrOZoCHxABJwK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,2021-11-06,2021,24,Jack Harlow,/wiki/Industry_Baby,Industry Baby,/wiki/Jack_Harlow,"(D-D-Daytrip took it to ten, hey)\nBaby back, ...",spotify:track:27NovPIUIRrOZoCHxABJwK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,2021-11-06,2021,27,Walker Hayes,/wiki/Fancy_Like,Fancy Like,/wiki/Walker_Hayes,"Ayy\nMy girl is bangin', she's so low maintena...",spotify:track:58UKC45GPNTflCN6nwCUeF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55739,2000-01-01,48,Cowboy Take Me Away,Dixie Chicks,52.0,48,6,2000-01-01,2000,95,Dixie Chicks,/wiki/Cowboy_Take_Me_Away,Cowboy Take Me Away,/wiki/Dixie_Chicks,I said I wanna touch the earth\nI wanna break ...,spotify:track:3rXCZRMiMZp0feGcYXpwYX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55740,2000-01-01,50,My Best Friend,Tim McGraw,54.0,50,6,2000-01-01,2000,83,Tim McGraw,/wiki/My_Best_Friend_(Tim_McGraw_song),My Best Friend,/wiki/Tim_McGraw,"I never had no one, that I could count on\nI'v...",spotify:track:5BeY2wY4rGFD1Q3aXO83Bv,0.444,0.484,9.0,-8.311,1.0,0.0289,0.120000,0.000000,0.135,0.263,147.715,audio_features,5BeY2wY4rGFD1Q3aXO83Bv,https://api.spotify.com/v1/tracks/5BeY2wY4rGFD...,https://api.spotify.com/v1/audio-analysis/5BeY...,279867.0,4.0
55741,2000-01-01,56,Say My Name,Destiny's Child,83.0,56,2,2000-01-01,2000,6,Destiny's Child,/wiki/Say_My_Name,Say My Name,/wiki/Destiny%27s_Child,"Darkchild '99\n\nSay my name, say my name\nIf ...",spotify:track:7H6ev70Weq6DdpZyyTmUXk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55742,2000-01-01,61,Higher,Creed,61.0,52,17,2000-01-01,2000,11,Creed,/wiki/Higher_(Creed_song),Higher,/wiki/Creed_(band),When dreaming I'm guided to another world\nTim...,spotify:track:1ZozJfi8u9cO2Ob8KwiwNT,0.459,0.830,2.0,-6.254,1.0,0.0364,0.000051,0.000140,0.206,0.431,155.827,audio_features,1ZozJfi8u9cO2Ob8KwiwNT,https://api.spotify.com/v1/tracks/1ZozJfi8u9cO...,https://api.spotify.com/v1/audio-analysis/1Zoz...,316733.0,4.0


In [38]:
artists = sorted(charts_and_lyrics_df['artist'].astype(str).unique().tolist())
artists

['"Groove" Holmes',
 '"Little" Jimmy Dickens',
 '"Pookie" Hudson',
 '"Weird Al" Yankovic',
 "'N Sync",
 "'N Sync & Gloria Estefan",
 "'N Sync Featuring Nelly",
 "'Til Tuesday",
 '(+44)',
 '(The Preacher) Bobby Womack',
 '1 Of The Girls',
 '10 Years',
 '10,000 Maniacs',
 '100 Proof Aged in Soul',
 '100 Strings and Jono (Choir of 40 Voices)',
 '10cc',
 '112',
 '112 Featuring Foxy Brown',
 "112 Featuring Lil'z",
 '112 Featuring Ludacris',
 '112 Featuring Mase',
 '112 Featuring Super Cat',
 '112 Featuring The Notorious B.I.G.',
 '12 Gauge',
 '1910 Fruitgum Co.',
 '1927',
 '2 Chainz',
 '2 Chainz & Wiz Khalifa',
 '2 Chainz Featuring Ariana Grande',
 '2 Chainz Featuring Drake',
 '2 Chainz Featuring Drake & Lil Wayne',
 '2 Chainz Featuring Kanye West',
 '2 Chainz Featuring Kendrick Lamar',
 '2 Chainz Featuring Lil Wayne',
 '2 Chainz Featuring Pharrell',
 '2 Chainz Featuring Travis Scott',
 '2 Chainz Featuring Ty Dolla $ign, Trey Songz & Jhene Aiko',
 '2 Chainz Featuring YG & Offset',
 '2 Chain

In [52]:
lyrics_df['time_signature'].value_counts()

time_signature
4.0    469
3.0     12
5.0      4
1.0      1
Name: count, dtype: int64

In [54]:
lyrics_df[lyrics_df['time_signature'] == 5.0]

,rank,song,band_singer,songurl,titletext,url,year,lyrics,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
134,16,Where the Party At,Jagged Edge,/wiki/Where_the_Party_At,Where the Party At,/wiki/Jagged_Edge_(American_group),2001,"Come on, come on, yeah\nCome on, yeah, come on...",spotify:track:5mNV8Mz59bzyuQ53gTw0c0,0.596,0.661,5.0,-6.239,0.0,0.226,0.31000,0.00000,0.0847,0.8600,129.491,audio_features,5mNV8Mz59bzyuQ53gTw0c0,https://api.spotify.com/v1/tracks/5mNV8Mz59bzy...,https://api.spotify.com/v1/audio-analysis/5mNV...,232573.0,5.0
137,17,Angel,Rayvon,/wiki/Angel_(Shaggy_song),Angel,/wiki/Rayvon,2001,Now this one's dedicated to all the youths who...,spotify:track:2Ei4V1qAiO2B1s4xtl7cl9,0.238,0.119,10.0,-13.600,0.0,0.038,0.95500,0.00133,0.1140,0.0626,80.639,audio_features,2Ei4V1qAiO2B1s4xtl7cl9,https://api.spotify.com/v1/tracks/2Ei4V1qAiO2B...,https://api.spotify.com/v1/audio-analysis/2Ei4...,225595.0,5.0
290,34,Just a Friend 2002,Mario,/wiki/Just_a_Friend_2002,Just a Friend 2002,/wiki/Mario_(entertainer),2002,You ready for me?\n\nI wanna know your name an...,spotify:track:2cxbxpHrND6i4uvUGVvC9J,0.538,0.677,0.0,-6.099,1.0,0.187,0.00541,0.00000,0.2360,0.9130,132.346,audio_features,2cxbxpHrND6i4uvUGVvC9J,https://api.spotify.com/v1/tracks/2cxbxpHrND6i...,https://api.spotify.com/v1/audio-analysis/2cxb...,214200.0,5.0
532,10,Lean Back,Fat Joe,/wiki/Lean_Back,Lean Back,/wiki/Fat_Joe,2004,"Feel this shit right here, Scott Storch, nigga...",spotify:track:2uiEqPjMmIJXOKplzsEASP,0.611,0.754,1.0,-5.035,0.0,0.366,0.11600,0.00000,0.0945,0.5650,126.961,audio_features,2uiEqPjMmIJXOKplzsEASP,https://api.spotify.com/v1/tracks/2uiEqPjMmIJX...,https://api.spotify.com/v1/audio-analysis/2uiE...,290267.0,5.0


In [56]:
#pip install billboard.py

In [61]:
url = 'https://www.billboard.com/charts/hot-100/'
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [62]:
titles = [tag.get_text(strip=True) for tag in soup.select('li ul li h3')]
artists = [tag.get_text(strip=True) for tag in soup.select('li ul li span.c-label') if not tag.attrs.get('class', [])[-1].startswith('c-label--chart')]

# Clean and zip
hot_100_songs = list(zip(titles[:100], artists[:100]))  # top 100 only

# Preview
for i, (title, artist) in enumerate(hot_100_songs[:5], start=1):
    print(f"{i}. {title} by {artist}")

1. Ordinary by Alex Warren
2. What I Want by 4
3. Just In Case by 1
4. I'm The Problem by 16
5. Luther by 4
